In [1]:
import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

import os
os.chdir("/home/sambeett2019/.kaggle/competitions/avito-demand-prediction")
print(os.listdir('/home/sambeett2019/.kaggle/competitions/avito-demand-prediction'))
import sys
reload(sys)
sys.setdefaultencoding('utf-8')

['user_category_lda_model.expElogbeta.npy', 'avg_price_user_type.csv', 'user_category_lda_model', 'periods_test.pkl', 'periods_train.pkl', 'use_features.csv', 'periods_test.csv', 'user_category_lda_model_10p.state', 'sample_submission.csv', 'user_item_corpus.mm', 'avg_price_parent_category.csv', 'user_item_dict', 'train_active.pkl', 'user_category_lda_model.state', 'train_norm.csv', 'avg_price_category.csv', 'test_active.csv', 'user_item_dict_filt', 'user_category_lda_model.id2word', 'avg_price_region.csv', 'user_category_lda_model_10p.id2word', 'user_category_lda_model_10p.expElogbeta.npy', 'test_jpg.zip', 'periods_train.csv', 'test_active.pkl', 'train_active.csv', 'user_preference_features.csv', 'avg_price_image_top_1.csv', 'test.pkl', 'train.csv', 'user_cat_lda_5topics.csv', 'train.pkl', 'user_category_lda_model_10p', 'train_jpg.zip', 'test.csv', 'test_norm.csv', 'avg_price_city.csv', 'user_item_corpus.mm.index', 'all_concat.pkl']


In [2]:
import gc
from tqdm import tqdm
import matplotlib.pyplot as plt
import seaborn as sns
from matplotlib_venn import venn2, venn2_circles
import string
from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_squared_error
from sklearn.preprocessing import LabelEncoder
from sklearn.feature_extraction.text import CountVectorizer, TfidfVectorizer
import nltk
from nltk.corpus import stopwords
import scipy
import lightgbm as lgb
from math import sqrt
from sklearn.linear_model import Ridge
from sklearn.cross_validation import KFold
from sklearn import feature_selection
from sklearn.pipeline import FeatureUnion

sns.set()
%matplotlib inline

In [3]:
SEED = 25
NFOLDS = 5

In [4]:
train = pd.read_csv('train_norm.csv',index_col = "item_id", parse_dates = ["activation_date"])
test = pd.read_csv('test_norm.csv', index_col = "item_id", parse_dates = ["activation_date"])

In [5]:
user_feat = pd.read_csv('use_features.csv')

In [6]:
print(os.listdir('/home/sambeett2019/.kaggle/competitions/avito-demand-prediction'))

In [7]:
#Price features
avg_price_category = pd.read_csv('avg_price_category.csv')
avg_price_parent_category = pd.read_csv('avg_price_parent_category.csv')
avg_price_region = pd.read_csv('avg_price_region.csv')
avg_price_city = pd.read_csv('avg_price_city.csv')
avg_price_user_type = pd.read_csv('avg_price_user_type.csv')
avg_price_image_top_1 = pd.read_csv('avg_price_image_top_1.csv')

In [8]:
#Preference features
user_preference = pd.read_csv('user_preference_features.csv')

In [9]:
user_preference.shape

(1009909, 9)

In [10]:
#LDA features
user_cat_lda_features = pd.read_csv('user_cat_lda_5topics.csv')

In [11]:
user_cat_lda_features.shape

(4361601, 6)

In [12]:
train.shape

(1503424, 17)

In [13]:
train.columns

Index([u'user_id', u'region', u'city', u'parent_category_name',
       u'category_name', u'param_1', u'param_2', u'param_3', u'title',
       u'description', u'price', u'item_seq_number', u'activation_date',
       u'user_type', u'image', u'image_top_1', u'deal_probability'],
      dtype='object')

In [14]:
train = train.merge(user_feat, on='user_id', how='left')
test = test.merge(user_feat, on='user_id', how='left')

In [15]:
train = train.merge(user_preference, on='user_id', how='left')
test = test.merge(user_preference, on='user_id', how='left')

In [16]:
train = train.merge(user_cat_lda_features, on='user_id', how='left')
test = test.merge(user_cat_lda_features, on='user_id', how='left')

In [17]:
train = train.merge(avg_price_category, on='category_name', how='left')
test = test.merge(avg_price_category, on='category_name', how='left')

In [18]:
train = train.merge(avg_price_parent_category, on='parent_category_name', how='left')
test = test.merge(avg_price_parent_category, on='parent_category_name', how='left')

In [19]:
train = train.merge(avg_price_region, on='region', how='left')
test = test.merge(avg_price_region, on='region', how='left')

In [20]:
train = train.merge(avg_price_city, on='city', how='left')
test = test.merge(avg_price_city, on='city', how='left')

In [21]:
train = train.merge(avg_price_user_type, on='user_type', how='left')
test = test.merge(avg_price_user_type, on='user_type', how='left')

In [22]:
train = train.merge(avg_price_image_top_1, on='image_top_1', how='left')
test = test.merge(avg_price_image_top_1, on='image_top_1', how='left')

In [23]:
ntrain = train.shape[0]
ntest = test.shape[0]

kf = KFold(ntrain, n_folds=5, shuffle=True, random_state=SEED)
y = train.deal_probability.copy()

In [24]:
count = lambda l1,l2: sum([1 for x in l1 if x in l2])
agg_cols = list(user_feat.columns)[1:]

In [25]:
del user_feat, user_preference, user_cat_lda_features, avg_price_category, avg_price_city, avg_price_image_top_1, avg_price_parent_category, avg_price_region, avg_price_user_type 
gc.collect()

133

In [26]:
stop_words= set(stopwords.words('russian'))

In [27]:
count = lambda l1,l2: sum([1 for x in l1 if x in l2])

for df in [train, test]:

    df['description'].fillna('unknowndescription', inplace=True)
    df['title'].fillna('unknowntitle', inplace=True)
    df['param_1'].fillna('unknownparam1', inplace=True)
    df['param_2'].fillna('unknownparam2', inplace=True)
    df['param_3'].fillna('unknownparam3', inplace=True)
   
    df['weekday'] = pd.to_datetime(df['activation_date']).dt.day
    df["Weekd of Year"] = pd.to_datetime(df['activation_date']).dt.week
    df["Day of Month"] = pd.to_datetime(df['activation_date']).dt.day

    for col in ['description', 'title']:
        df['num_chars_' + col] = df[col].apply(len)
        df['num_words_' + col] = df[col].apply(lambda comment: len(comment.split()))
        df['num_unique_words_' + col] = df[col].apply(lambda comment: len(set(w for w in comment.split())))
#        df['title_word_coun_' + col] = df['description'].apply(lambda x: len([wrd for wrd in x.split() if wrd.istitle()]))
#        df['upper_case_word_count_' + col] = df['description'].apply(lambda x: len([wrd for wrd in x.split() if wrd.isupper()]))
#        df['stopword_count_' + col] = df['description'].apply(lambda x: len([wrd for wrd in x.split() if wrd.lower() in stop_words ]))
#        df['num_desc_punct_' + col] = df['description'].apply(lambda x: count(x, set(string.punctuation)))

    df['words_vs_unique_title'] = df['num_unique_words_title'] / df['num_words_title'] * 100
    df['words_vs_unique_description'] = df['num_unique_words_description'] / df['num_words_description'] * 100

    df['city'] = df['region'] + '_' + df['city']              ##combine city and region
    
    for col in agg_cols:
        df[col].fillna(-1, inplace=True)
    df["price"] = np.log(df["price"]+0.001)
    df["price"].fillna(-999,inplace=True)

    

Tf - IDF

In [28]:
count_vectorizer_title = CountVectorizer(stop_words=stopwords.words('russian'), lowercase=True, min_df=10)

title_counts = count_vectorizer_title.fit_transform(train['title'].append(test['title']))

train_title_counts = title_counts[:len(train)]
test_title_counts = title_counts[len(train):]


count_vectorizer_desc = TfidfVectorizer(stop_words=stopwords.words('russian'), 
                                        lowercase=True, ngram_range=(1, 2),
                                        max_features=18000)

desc_counts = count_vectorizer_desc.fit_transform(train['description'].append(test['description']))

train_desc_counts = desc_counts[:len(train)]
test_desc_counts = desc_counts[len(train):]

train_title_counts.shape, train_desc_counts.shape

((1503424, 24392), (1503424, 18000))

In [29]:
ready_train = scipy.sparse.hstack([ train_desc_counts,train_title_counts], format='csr')
ready_test = scipy.sparse.hstack([ test_desc_counts,test_title_counts], format='csr')

In [30]:
target = 'deal_probability'
categorical = ['image_top_1', 'param_1', 'param_2', 'param_3', 'city', 'region', 'category_name', 'parent_category_name',
               'user_type','preferred_category','preferred_parent_category','preferred_region','preferred_city']

In [31]:
for feature in categorical:
    print 'Transforming {feature}...'
    encoder = LabelEncoder()
    encoder.fit(train[feature].append(test[feature]).astype(str))
    
    train[feature] = encoder.transform(train[feature].astype(str))
    test[feature] = encoder.transform(test[feature].astype(str))

In [32]:
class SklearnWrapper(object):
    def __init__(self, clf, seed=0, params=None, seed_bool = True):
        if(seed_bool == True):
            params['random_state'] = seed
        self.clf = clf(**params)

    def train(self, x_train, y_train):
        self.clf.fit(x_train, y_train)

    def predict(self, x):
        return self.clf.predict(x)
        
def get_oof(clf, x_train, y, x_test):
    oof_train = np.zeros((ntrain,))
    oof_test = np.zeros((ntest,))
    oof_test_skf = np.empty((NFOLDS, ntest))

    for i, (train_index, test_index) in enumerate(kf):
        print('\nFold {}'.format(i))
        x_tr = x_train[train_index]
        y_tr = y[train_index]
        x_te = x_train[test_index]

        clf.train(x_tr, y_tr)

        oof_train[test_index] = clf.predict(x_te)
        oof_test_skf[i, :] = clf.predict(x_test)

    oof_test[:] = oof_test_skf.mean(axis=0)
    return oof_train.reshape(-1, 1), oof_test.reshape(-1, 1)

In [33]:
ridge_params = {'alpha':20.0, 'fit_intercept':True, 'normalize':False, 'copy_X':True,
                'max_iter':None, 'tol':0.001, 'solver':'auto', 'random_state':SEED}

In [34]:
ridge = SklearnWrapper(clf=Ridge, seed = SEED, params = ridge_params)
ridge_oof_train, ridge_oof_test = get_oof(ridge, ready_train, y, ready_test)

In [35]:
train['ridge_preds'] = ridge_oof_train
test['ridge_preds'] = ridge_oof_test

In [36]:
remove_feats = ['title','description','activation_date','image','user_id']
train.drop(remove_feats, axis=1,inplace=True)
test.drop(remove_feats, axis=1,inplace=True)

In [37]:
predictors = list(train.columns.values)

In [38]:
predictors.remove('deal_probability')

In [39]:
predictors

['region',
 'city',
 'parent_category_name',
 'category_name',
 'param_1',
 'param_2',
 'param_3',
 'price',
 'item_seq_number',
 'user_type',
 'image_top_1',
 'avg_days_up_user',
 'avg_times_up_user',
 'n_user_items',
 'n_items_per_day',
 'preferred_category',
 'preferred_parent_category',
 'preferred_region',
 'preferred_city',
 'unique_categories',
 'unique_parent_categories',
 'unique_regions',
 'unique_cities',
 'user_cat_topic_1',
 'user_cat_topic_2',
 'user_cat_topic_3',
 'user_cat_topic_4',
 'user_cat_topic_5',
 'avg_price_category',
 'avg_price_parent_category',
 'avg_price_region',
 'avg_price_city',
 'avg_price_user_type',
 'avg_price_image_top_1',
 'weekday',
 'Weekd of Year',
 'Day of Month',
 'num_chars_description',
 'num_words_description',
 'num_unique_words_description',
 'num_chars_title',
 'num_words_title',
 'num_unique_words_title',
 'words_vs_unique_title',
 'words_vs_unique_description',
 'ridge_preds']

LGB Model

In [40]:
rounds = 16000
early_stop_rounds = 30
params = {
    'boosting_type': 'gbdt',
    'objective' : 'regression',
    'metric' : 'rmse',
    'num_leaves' : 250,
    'max_depth': 15,
    'learning_rate' : 0.02,
    'feature_fraction' : 0.7,
    'bagging_fraction' : 0.85,
    'verbosity' : -1
}
feature_names = np.hstack([
    count_vectorizer_desc.get_feature_names(),
    count_vectorizer_title.get_feature_names(),
    predictors
])
print('Number of features:', len(feature_names))


In [41]:
train_index, valid_index = train_test_split(np.arange(len(train)), test_size=0.1, random_state=42)

x_train = scipy.sparse.hstack([
        train_desc_counts[train_index],
        train_title_counts[train_index],
        train.loc[train_index, predictors]
], format='csr')
y_train = train.loc[train_index, target]

x_valid = scipy.sparse.hstack([
    train_desc_counts[valid_index],
    train_title_counts[valid_index],
    train.loc[valid_index, predictors]
], format='csr')
y_valid = train.loc[valid_index, target]

x_test = scipy.sparse.hstack([
    test_desc_counts,
    test_title_counts,
    test.loc[:, predictors]
], format='csr')

dtrain = lgb.Dataset(x_train, label=y_train,
                     feature_name=list(feature_names), 
                     categorical_feature=categorical)
dvalid = lgb.Dataset(x_valid, label=y_valid,
                     feature_name=list(feature_names), 
                     categorical_feature=categorical)

In [43]:
evals_result = {}
model = lgb.train(params, dtrain, 
                  valid_sets=[dtrain, dvalid], 
                  valid_names=['train', 'valid'],
                  num_boost_round=rounds, 
                  early_stopping_rounds=early_stop_rounds, 
                  verbose_eval=100)
model.save('abcd')

LightGBMError: Cannot set categorical feature after freed raw data, set free_raw_data=False when construct Dataset to avoid this.

In [ ]:
fig, ax = plt.subplots(figsize=(10, 14))
lgb.plot_importance(model, max_num_features=50, ax=ax)
plt.title("Light GBM Feature Importance")

In [ ]:
subm = pd.read_csv('sample_submission.csv')
subm['deal_probability'] = np.clip(model.predict(x_test), 0, 1)
subm.to_csv('lgb_submission.csv', index=False)

In [ ]:
ridge_pred = pd.DataFrame(ridge_oof_test[:,0])
ridge_pred.to_csv('ridge_pred.csv', index=False)